In [1]:
import numpy as np
import scipy.special
from scipy.integrate import trapezoid
from numpy import cos, sin
from numpy.linalg import norm

In [2]:
# Mathieu functions with radian input
def mathieu_cem(m, q, x):
    return scipy.special.mathieu_cem(m, q, 180/np.pi*x)
def mathieu_sem(m, q, x):
    return scipy.special.mathieu_sem(m, q, 180/np.pi*x)

In [3]:
def se_q_expansion(m,q,v):
  # This implements the general expansion for small q shown
  # in DLMF 28.6.26
    
  t1 = sin(m*v)
  t2 = q*(sin((m+2)*v)/(m+1) - sin((m-2)*v)/(m-1))/4
  t3 = (q**2)*(sin((m+4)*v)/((m+1)*(m+2)) + sin((m-4)*v)/((m-1)*(m-2)) - 2*(m*m+1)/((m*m-1)^2)*sin(m*v))/32
  ce = t1-t2+t3
  return ce

In [4]:
# This checks se using a few identities.
    
fail = 0
    
qs = np.logspace(-3,3,21)
qs = np.hstack([-qs, qs])
#qs = [-1000, -100, =10, =1, -0.1, -0.01, -0.001, 0, 0.0001, .001, .01, .1, 1, 10, 100, 1000];
#qs = [1, 10];    

N = 1000
v = np.linspace(-np.pi,np.pi,N)



In [5]:
MM = 10
tol = 1e-13
for m in range(1,MM+1):
    print(f'-----------  m = {m} -----------\n')
    for i,q in enumerate(qs):
        
        se,_ = mathieu_sem(m,q,v)
        s = trapezoid(se*se, v)
        
        diff = s - np.pi
        print(f's = {s}, err = {diff}\n')
        if (np.abs(diff) > tol):
            print(f'Error!  m = {m}, q = {q}, diff = {diff}\n')
            fail = fail+1


-----------  m = 1 -----------

s = 3.1415926535897922, err = -8.881784197001252e-16

s = 3.1415926535897927, err = -4.440892098500626e-16

s = 3.1415926535897922, err = -8.881784197001252e-16

s = 3.1415926535897905, err = -2.6645352591003757e-15

s = 3.141592653589793, err = 0.0

s = 3.141592653589793, err = 0.0

s = 3.141592653589793, err = 0.0

s = 3.1415926535897922, err = -8.881784197001252e-16

s = 3.1415926535897913, err = -1.7763568394002505e-15

s = 3.1415926535897927, err = -4.440892098500626e-16

s = 3.1415926535897922, err = -8.881784197001252e-16

s = 3.141592653589793, err = 0.0

s = 3.141592653589792, err = -1.3322676295501878e-15

s = 3.1415926535897927, err = -4.440892098500626e-16

s = 3.1415926535897922, err = -8.881784197001252e-16

s = 3.1415926535897936, err = 4.440892098500626e-16

s = 3.1415926535897927, err = -4.440892098500626e-16

s = 3.1415926535897922, err = -8.881784197001252e-16

s = 3.141592653589793, err = 0.0

s = 3.1415926535897856, err = -7.54951656

In [6]:
from itertools import combinations
from scipy.integrate import quad
#====================================================
# Next test orthogonality per DLMF 28.2,32
print('Testing orthogonality DLMF 28.2,32 ... \n')
tol = 1e-11
MM = 5
for m1, m2 in combinations(range(1,MM+1),2):
    print(f'-----------  [m1,m2] = [{m1},{m2}]  -----------\n')
    for i,q in enumerate(qs):
          
        se = lambda m,q,v: mathieu_sem(m,q,v)[0] 
        f = lambda v: se(m1,q,v)*se(m2,q,v)
        s = quad(f, -np.pi, np.pi)[0]
        
        diff = s
        print(f's = {s}, err = {diff}\n')
        if (abs(diff) > tol):
            print(f'Error!  [m1,m2] = [{m1},{m2}], q = {q}, diff = {diff}\n')
            fail = fail+1    


Testing orthogonality DLMF 28.2,32 ... 

-----------  [m1,m2] = [1,2]  -----------

s = 4.176530142471329e-17, err = 4.176530142471329e-17

s = 2.0035478050595917e-17, err = 2.0035478050595917e-17

s = 1.3130764832057332e-17, err = 1.3130764832057332e-17

s = 2.2916550155591945e-17, err = 2.2916550155591945e-17

s = 1.799398698887752e-17, err = 1.799398698887752e-17

s = 3.132633656922098e-17, err = 3.132633656922098e-17

s = 4.3308803486006584e-17, err = 4.3308803486006584e-17

s = 2.925728085950548e-17, err = 2.925728085950548e-17

s = 7.121567348751471e-18, err = 7.121567348751471e-18

s = 4.99364597923091e-17, err = 4.99364597923091e-17

s = 1.032113059928032e-16, err = 1.032113059928032e-16

s = 6.461364076024061e-17, err = 6.461364076024061e-17

s = -2.08226899113128e-16, err = -2.08226899113128e-16

s = -6.643533418307798e-16, err = -6.643533418307798e-16

s = -1.473974518760072e-15, err = -1.473974518760072e-15

s = -2.330764538773477e-15, err = -2.330764538773477e-15

s = -2.2

In [7]:
# ====================================================
# Test q = 0 case per DLMF 28.2.29
print(f'Test se tends to sin for q = -1e-13, 0.0, 1e-13 ... \n')
tol = 5e-13
q_test = [-1e-13, 0.0, 1e-13]
MM = 10  # Max order to test
for q in q_test:
  print(f'=========== q = {q}  ===========\n')
  for m in range(1,MM+1):
    print(f'-----------  m = {m}  -----------\n')
    LHS, _ = mathieu_sem(m,q,v)
    RHS = sin(m*v)
    
    ndiff = norm(LHS-RHS)
    print(f'm = {m}, LHS(1) = {LHS[0]}, RHS(1) = {RHS[0]}, norm err = {ndiff}\n')
    if (ndiff > tol):
      print(f'Error!  m = {m}, q = {q}, LHS(1) = {LHS[0]}, RHS(1) = {RHS[0]}, ndiff = {ndiff}\n')
      fail = fail+1


Test se tends to sin for q = -1e-13, 0.0, 1e-13 ... 

=========== q = -1e-13  ===========

-----------  m = 1  -----------

m = 1, LHS(1) = 7.044813998280375e-16, RHS(1) = -1.2246467991473532e-16, norm err = 2.7602142110085455e-13

-----------  m = 2  -----------

m = 2, LHS(1) = -1.4089627996560679e-15, RHS(1) = 2.4492935982947064e-16, norm err = 1.8135691893030362e-13

-----------  m = 3  -----------

m = 3, LHS(1) = 1.2252657797839602e-15, RHS(1) = -3.6739403974420594e-16, norm err = 3.291030478841634e-13

-----------  m = 4  -----------

m = 4, LHS(1) = -2.8179255993120893e-15, RHS(1) = 4.898587196589413e-16, norm err = 2.1093618902899692e-13

-----------  m = 5  -----------

m = 5, LHS(1) = 4.41058541884026e-15, RHS(1) = -6.123233995736766e-16, norm err = 1.6830013596517163e-13

-----------  m = 6  -----------

m = 6, LHS(1) = -2.450531559567889e-15, RHS(1) = 7.347880794884119e-16, norm err = 1.3796058945892693e-13

-----------  m = 7  -----------

m = 7, LHS(1) = 7.59590505789652

In [8]:
#====================================================
# Next test even fcns per DLMF 28.2.34
print('Test rotation identity for even fcns per DLMF 28.2,34 ... \n')
tol = 1e-13
v = 0.05  # Just check one random point.
MM = 10;  # Sets max order to test
for m in range(2, MM+1, 2):
    ss = ((-1)**((m-2)/2))
    print(f'-----------  m = {m}, ss = {ss}  -----------\n')
    print(f's = {s}\n')
    for i, q in enumerate(qs):
        LHS, _ = mathieu_sem(m,-q,v)
        RHS = ss*mathieu_sem(m,q,(np.pi/2)-v)[0]
        diff = LHS-RHS
        print(f'm = {m}, LHS = {LHS}, RHS = {RHS}, err = {diff}\n')
        if (abs(diff) > tol):
            print(f'Error!  m = {m}, q = {q}, LHS = {LHS}, RHS = {RHS}, diff = {diff}\n')
            fail = fail+1

  
print('======================================\n')  
 

Test rotation identity for even fcns per DLMF 28.2,34 ... 

-----------  m = 2, ss = 1.0  -----------

s = -9.034422368275974e-15

m = 2, LHS = 0.09981686129890137, RHS = 0.09981686129890155, err = -1.8041124150158794e-16

m = 2, LHS = 0.09980038522561639, RHS = 0.09980038522561657, err = -1.8041124150158794e-16

m = 2, LHS = 0.09976751361943796, RHS = 0.09976751361943814, err = -1.8041124150158794e-16

m = 2, LHS = 0.09970193617319864, RHS = 0.09970193617319882, err = -1.8041124150158794e-16

m = 2, LHS = 0.0995711319404081, RHS = 0.09957113194040829, err = -1.8041124150158794e-16

m = 2, LHS = 0.09931030348414611, RHS = 0.0993103034841463, err = -1.8041124150158794e-16

m = 2, LHS = 0.09879052965249283, RHS = 0.098790529652493, err = -1.8041124150158794e-16

m = 2, LHS = 0.09775609445137141, RHS = 0.0977560944513716, err = -1.8041124150158794e-16

m = 2, LHS = 0.09570324796012014, RHS = 0.09570324796012034, err = -1.942890293094024e-16

m = 2, LHS = 0.09165603337032438, RHS = 0.09165

In [9]:
#====================================================
# Next test odd fcns per DLMF 28.2,35
print(f'Test rotation identity for odd fcns per DLMF 28.2,35 ... \n')
v = 0.05;  # Just check one random point.
MM = 10;  # Sets max order to test
for m in range(1, MM+2, 2):
    ss = ((-1)**((m-1)/2))
    print(f'-----------  m = {m}, ss = {ss} -----------\n')
    print(f's = {s}\n')
    for i,q in enumerate(qs):
      LHS = mathieu_sem(m,-q,v)[0]
      RHS = ss*mathieu_cem(m,q,np.pi/2-v)[0]
      diff = LHS-RHS
      print(f'm = {m}, LHS = {LHS}, RHS = {RHS}, err = {diff}\n')
      if (abs(diff) > tol):
        print(f'Error!  m = {m}, q = {q}, LHS = {LHS}, RHS = {RHS}, diff = {diff}\n')
        fail = fail+1

print('======================================\n')

Test rotation identity for odd fcns per DLMF 28.2,35 ... 

-----------  m = 1, ss = 1.0 -----------

s = -9.034422368275974e-15

m = 1, LHS = 0.049960492737087726, RHS = 0.04996049273708782, err = -9.020562075079397e-17

m = 1, LHS = 0.049941911106658396, RHS = 0.049941911106658486, err = -9.020562075079397e-17

m = 1, LHS = 0.04990485501458898, RHS = 0.04990485501458907, err = -9.020562075079397e-17

m = 1, LHS = 0.04983099455032861, RHS = 0.0498309945503287, err = -9.020562075079397e-17

m = 1, LHS = 0.04968392660580912, RHS = 0.04968392660580921, err = -9.020562075079397e-17

m = 1, LHS = 0.04939169281669436, RHS = 0.04939169281669445, err = -9.020562075079397e-17

m = 1, LHS = 0.04881339868642969, RHS = 0.04881339868642978, err = -9.020562075079397e-17

m = 1, LHS = 0.047678531965402254, RHS = 0.047678531965402345, err = -9.020562075079397e-17

m = 1, LHS = 0.04548897629799342, RHS = 0.045488976297993505, err = -8.326672684688674e-17

m = 1, LHS = 0.04141050064941937, RHS = 0.04141

In [10]:
#====================================================
# Next test small q expansions per DLMF 28.6.23.  The goal
# is to make sure my fcn impls go the right way for negative
# q values.
print('Test small q expansions per DLMF 28.6.23 ... \n')

tol = 1e-4
N = 1000
v = np.linspace(-np.pi,np.pi,N)

qs = [-1.5, -1, -.5, -.2, -.1, .1, .2, .5, 1, 1.5]

MM = 35

# The first three orders don't work unless q is really small
#-----------------------------------------------------------
m = 1
qs = [-.2, -.1, .1, .2];  
for i, q in enumerate(qs):
    # Define power series after setting q.
    se1 = lambda v: sin(v) - q*sin(3*v)/8 + (q**2)*(2*sin(5*v)/3 + 2*sin(3*v) - sin(v))/128 - (q**3)*(sin(7*v)/9 + 8*sin(5*v)/9 - sin(3*v)/3 - 2*sin(v))/1024
    
    print(f'----------- m = {m}, q = {q}  -----------\n')  
    dlmfse1 = se1(v)
    myse1, _ = mathieu_sem(m,q,v)
    # %plot(v, dlmfse1,'b-')
    # %hold on
    # %plot(v, myse1,'r.')
    # %legend('dlmf','me')
    ndiff = norm(dlmfse1 - myse1)/N
    if (ndiff > tol):
        print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n')
        fail = fail+1


Test small q expansions per DLMF 28.6.23 ... 

----------- m = 1, q = -0.2  -----------

----------- m = 1, q = -0.1  -----------

----------- m = 1, q = 0.1  -----------

----------- m = 1, q = 0.2  -----------



In [11]:
#-----------------------------------------------------------  
# m = 2
m = 2
qs = [-.5, -.2, -.1, .1, .2, .5];    
for i,q  in enumerate(qs):
  # Define power series after setting q.
  se2 = lambda v: sin(2*v) - q*sin(4*v)/12 + (q**2)*(sin(6*v)/3 - 4*sin(2*v)/9)/128
  print(f'----------- m = {m}, q = {q}  -----------\n')  
  dlmfse2 = se2(v)
  myse2, _ = mathieu_sem(m,q,v)
  # %plot(v, dlmfse2)
  # %hold on
  # %plot(v, myse2)
  # %legend('dlmf','me')
  ndiff = norm(dlmfse2 - myse2)/N
  if (ndiff > tol):
    print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n')
    fail = fail+1


----------- m = 2, q = -0.5  -----------

----------- m = 2, q = -0.2  -----------

----------- m = 2, q = -0.1  -----------

----------- m = 2, q = 0.1  -----------

----------- m = 2, q = 0.2  -----------

----------- m = 2, q = 0.5  -----------



In [12]:
#-----------------------------------------------------------  
# Higher orders work over larger q domains
# Now m = 3, 4, 5, ... 11
qs = [-1.5, -1, -.5, -.1, .1, .5, 1, 1.5];  
for i, q in enumerate(qs):
    for m in range(3, 11+1):
      print(f'----------- m = {m}, q = {q}  -----------\n')  
      dlmfse = se_q_expansion(m,q,v)
      myse, _ = mathieu_sem(m,q,v)
      # %plot(v, dlmfse,'b-')
      # %hold on
      # %plot(v, myse,'r.')
      # %legend('dlmf','me')
      ndiff = norm(dlmfse - myse)/N
      if (ndiff > tol):
        print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n')
        fail = fail+1

----------- m = 3, q = -1.5  -----------

Error!  m = 3, q = -1.5, ndiff = 0.0025108140574311246

----------- m = 4, q = -1.5  -----------

Error!  m = 4, q = -1.5, ndiff = 0.0038777228048679437

----------- m = 5, q = -1.5  -----------

Error!  m = 5, q = -1.5, ndiff = 0.0030009752284954755

----------- m = 6, q = -1.5  -----------

Error!  m = 6, q = -1.5, ndiff = 0.0034289864917830997

----------- m = 7, q = -1.5  -----------

Error!  m = 7, q = -1.5, ndiff = 0.00307473127997741

----------- m = 8, q = -1.5  -----------

Error!  m = 8, q = -1.5, ndiff = 0.0032975429001540395

----------- m = 9, q = -1.5  -----------

Error!  m = 9, q = -1.5, ndiff = 0.0031026453190916327

----------- m = 10, q = -1.5  -----------

Error!  m = 10, q = -1.5, ndiff = 0.0032401247510256935

----------- m = 11, q = -1.5  -----------

Error!  m = 11, q = -1.5, ndiff = 0.003116277924960816

----------- m = 3, q = -1  -----------

Error!  m = 3, q = -1, ndiff = 0.0011345102798999015

----------- m = 4, q = 

In [13]:
#-----------------------------------------------------------  
# Now even higher orders
qs = [-10, -3, -1.5, -1, -.5, -.1, .1, .5, 1, 1.5, 3, 10];  
for i, q in enumerate(qs):
    for m in range(12,MM+1):
      print(f'----------- m = {m}, q = {q}  -----------\n')  
      dlmfse = se_q_expansion(m,q,v)
      myse,_ = mathieu_sem(m,q,v)
      #plot(v, dlmfse,'b-')
      #hold on
      #plot(v, myse,'r.')
      #legend('dlmf','me')
      ndiff = norm(dlmfse - myse)/N
      if (ndiff > tol):
        print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n')
        fail = fail+1

----------- m = 12, q = -10  -----------

Error!  m = 12, q = -10, ndiff = 0.1426667695861662

----------- m = 13, q = -10  -----------

Error!  m = 13, q = -10, ndiff = 0.1388504636354295

----------- m = 14, q = -10  -----------

Error!  m = 14, q = -10, ndiff = 0.14186126095887647

----------- m = 15, q = -10  -----------

Error!  m = 15, q = -10, ndiff = 0.13905943742359955

----------- m = 16, q = -10  -----------

Error!  m = 16, q = -10, ndiff = 0.1413440089608537

----------- m = 17, q = -10  -----------

Error!  m = 17, q = -10, ndiff = 0.13919855337756704

----------- m = 18, q = -10  -----------

Error!  m = 18, q = -10, ndiff = 0.14099188983984273

----------- m = 19, q = -10  -----------

Error!  m = 19, q = -10, ndiff = 0.1392958461213296

----------- m = 20, q = -10  -----------

Error!  m = 20, q = -10, ndiff = 0.1407412654205845

----------- m = 21, q = -10  -----------

Error!  m = 21, q = -10, ndiff = 0.13936656089728044

----------- m = 22, q = -10  -----------

Err